This is a notebook with relevant scripts to transfrom and query data on SAC PM influence Model

Run block to Install pandas 

In [4]:
%pip install pandas
%pip install numpy
%pip install sentence_transformers
%pip install flupy tqdm python-dotenv
%pip install supabase
%pip install vecs
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached ipywidgets-8.1.3-py3-none-any.whl.metadata (2.4 kB)
  Using cached widgetsnbextension-4.0.11-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.11-py3-none-any.whl.metadata (4.1 kB)
Using cached ipywidgets-8.1.3-py3-none-any.whl (139 kB)
Using cached jupyterlab_widgets-3.0.11-py3-none-any.whl (214 kB)
Using cached widgetsnbextension-4.0.11-py3-none-any.whl (2.3 MB)
Note: you may need to restart the kernel to use updated packages.


Import modules

Supabase Credentials are saved as Environment Variables

In [3]:
import csv
import re
import os
import pandas as pd
from sentence_transformers import SentenceTransformer
import vecs
from flupy import flu
from tqdm import tqdm
from typing import List, Tuple, Dict
import numpy as np

import supabase
from sklearn.metrics.pairwise import cosine_similarity
import ast
from dotenv import load_dotenv


load_dotenv()


c:\Users\I586445\AppData\Local\anaconda3\envs\vecs\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


False

How to clean data from https://sapit-iex-prod-lizard-customer-influence-workbench-ciw-6f02fb4d.cfapps.eu10-004.hana.ondemand.com/cccissessionowner/index.html#/DetailView/293,I311255

Change CSV path and run 

In [3]:


def clean_data(df):
    # Drop columns: 'FBA Comments', 'FBA Status', 'User ID', 'Internal Project ID'
    df = df.drop(columns=['FBA Comments', 'FBA Status', 'User ID', 'Internal Project ID'])
    # Drop columns: 'Is Submitter ?', 'ERP Customer No.', 'Account Type', 'Voter Zip Code', 'Voter City', 'Voter Street', 'Voter Email', 'Voter Name'
    df = df.drop(columns=['Is Submitter ?', 'ERP Customer No.', 'Account Type', 'Voter Zip Code', 'Voter City', 'Voter Street', 'Voter Email', 'Voter Name'])
    # Drop columns: 'INM Project Name', 'Coach Name', 'Category'
    df = df.drop(columns=['INM Project Name', 'Coach Name', 'Category'])
    # Drop columns: 'IR Link', 'IR Title', 'Decision Reason', 'Reference'
    df = df.drop(columns=['IR Link', 'IR Title', 'Decision Reason', 'Reference'])
    # Drop columns: 'Views', 'Customer Votes', 'Last Status Change Date', 'Status', 'Reason Comments'
    df = df.drop(columns=['Views', 'Customer Votes', 'Last Status Change Date', 'Status', 'Reason Comments'])
    # Change column type to datetime64[ns] for column: 'Voted Date'
    df = df.astype({'Voted Date': 'datetime64[ns]'})
    # Change column type to string for column: 'Voter Company'
    df = df.astype({'Voter Company': 'string'})
    # Drop duplicate rows in columns: 'IR ID', 'Voter Company'
    df = df.drop_duplicates(subset=['IR ID', 'Voter Company'])
    # Change column type to category for column: 'Voter Company'
    df = df.astype({'Voter Company': 'category'})
    # Change column type to string for column: 'ir id'
    df = df.astype({'ir id': 'string'})
    return df

# Loaded variable 'df' from URI: 
df = pd.read_csv(r'c:\\Users\\I586445\\OneDrive - National University of Ireland, Galway\\Documents\\InfluenceScripts\\Copy of Improvements_and_Votes(18-06-2024, 13_35).csv')

df = clean_data(df)
#save to csv
df.to_csv(r'c:\\Users\\I586445\\OneDrive - National University of Ireland, Galway\\Documents\\InfluenceScripts\\IR_Votes_cleaned.csv', index=False)

How to create consumer Foreign keys to connect Jira to influence 

In [1]:


# Specify the path to your CSV file
csv_file_path = r"C:\Users\I586445\Downloads\17-06 Model Update\FPA34 17_06.csv"
# Construct the output file path
output_folder = os.path.dirname(csv_file_path)
output_file_path = os.path.join(output_folder, "Consumer Foreign keys.csv")

# Open the CSV file for reading and the output file for writing
with open(csv_file_path, 'r', encoding='utf-8-sig') as file, open(output_file_path, 'w', newline='', encoding='utf-8') as output_file:
    # Create a CSV reader object
    csv_reader = csv.reader(file)
    # Create a CSV writer object
    csv_writer = csv.writer(output_file)
    
    # Write the header row with selected columns
    csv_writer.writerow(["ISSUEKEY", "CONSUMER"])
    
    # Process the header row to get the column indices
    header = next(csv_reader)
    column_indices = {}
    for column_name in ["ISSUEKEY", "CONSUMER"]:
        try:
            column_indices[column_name] = header.index(column_name)
        except ValueError:
            # Handle the case if any of the required columns are not found in the header
            print(f"Column '{column_name}' not found in the CSV file.")
            exit(1)

    # Process each row and write selected columns to the output file
    for row in csv_reader:
        issue_key = row[column_indices["ISSUEKEY"]]
        consumer_digits = row[column_indices["CONSUMER"]]
        
    
        # Extract the "Consumer Digits" from the "Custom field (Consumer)" column using regex
        matches_consumer = re.findall(r"\bhttps://influence.sap.com.*?(\d{6})", consumer_digits)
        
        # If matches are found, create a new row for each match
        if matches_consumer:
            for match in matches_consumer:
                # Create a new row with details from the original row and the corresponding match
                new_row = [issue_key, match]
                csv_writer.writerow(new_row)
        else:
            # If no matches found, create a new row with details from the original row and an empty "Consumer Digits"
            new_row = [issue_key, ""]
            csv_writer.writerow(new_row)

print("Code executed successfully.")


Code executed successfully.


Searching for explicit mention of text in the descrtipon field of influence 

In [ ]:
# Paths to the CSV files
Influence_CSV = r"C:\Users\I586445\OneDrive - National University of Ireland, Galway\Documents\InfluenceScripts\simlarity\SAP Analytics Cloud & SAP Digital Boardroom.csv"
# Initialize influence to None or an empty DataFrame
influence = None

# Load CSV files into Pandas DataFrames
try:
    influence = pd.read_csv(Influence_CSV, usecols=['Request_ID', 'Idea_Title', 'Description', 'Status','Voting Score'])
except Exception as e:
    print(f"Error reading the CSV file: {e}")

# Proceed only if influence is not None
if influence is not None:
    # Search for text in the description column LIKE% BW Live or BW or LIVE or Business Warehouse
    search = influence[influence['Description'].str.contains('BW Live|BW|Business Warehouse', case=False, na=False)]

    # Save the filtered data to a new CSV file
    search.to_csv('search.csv', index=False)

    print("Data search completed successfully!")
else:
    print("Failed to load data. Please check the CSV file and column names.")


This  snippet reads data from a CSV file, checks each row for the presence of the string "SAC_PM_ACTIVE", and writes selected data along with the check result to a new CSV file. 

It specifically extracts "Issue key" and "Issue id" from the input, adds a boolean indicating the presence of "SAC_PM_ACTIVE", and leaves "Consumer Digits" empty in the output file.

In [ ]:

# Specify the path to your CSV file
csv_file_path = r"C:\Users\I586445\Downloads\model refresh\SAPJIRA without influence.csv"
# Construct the output file path
output_folder = os.path.dirname(csv_file_path)
output_file_path = os.path.join(output_folder, "SAC_PM_ACTIVE_NO_INFLUENCE.csv")

# Open the CSV file for reading and the output file for writing
with open(csv_file_path, 'r', encoding='utf-8') as file, open(output_file_path, 'w', newline='', encoding='utf-8') as output_file:
    # Create a CSV reader object
    csv_reader = csv.reader(file)
    # Create a CSV writer object
    csv_writer = csv.writer(output_file)
    
    # Write the header row with selected columns
    csv_writer.writerow(["Issue key", "Issue id", "SAC_PM_ACTIVE", "Consumer Digits"])
    
    # Process the header row to get the column indices
    header = next(csv_reader)
    column_indices = {}
    for column_name in ["Issue key", "Issue id"]:
        try:
            column_indices[column_name] = header.index(column_name)
        except ValueError:
            # Handle the case if any of the required columns are not found in the header
            print(f"Column '{column_name}' not found in the CSV file.")
            exit(1)

    # Process each row and write selected columns to the output file
    for row in csv_reader:
        issue_key = row[column_indices["Issue key"]]
        issue_id = row[column_indices["Issue id"]]
    
        
        # Check if "SAC_PM_ACTIVE" is found in any field of the row
        sac_pm_active = any("SAC_PM_ACTIVE" in field for field in row)
        
        new_row = [issue_key, issue_id, sac_pm_active, ""]
        csv_writer.writerow(new_row)
        

print("Code executed successfully.")

How to analyse influence requests via cosine Simlarity using Pgvector from supabase 

This requires supabase account 
https://supabase.com/docs/guides/ai

Converting Requests to Vectors 

In [ ]:


# Load the database connection string from the environment variables
DB_CONNECTION = os.getenv("DB_CONNECTION")

# Create vector store client
vx = vecs.create_client(DB_CONNECTION)
print("Connected to vector store")

# Create a PostgreSQL/pgvector table named "product_requests" to contain the request embeddings
product_requests = vx.get_or_create_collection(name="product_requests", dimension=384)

# Load the CSV file
csv_file = r"C:\Users\I586445\Downloads\improvement request_12-06-2024_11-48\SAP Analytics Cloud & SAP Digital Boardroom.csv"

def preprocess_description(description: str) -> str:
    # Sentences to exclude
    sentences_to_exclude = [
        "Please describe your improvement request",
        "What is the opportunity/problem the request will address?",
        "What is the expected benefit?"
    ]

    # Remove the sentences
    for sentence in sentences_to_exclude:
        description = description.replace(sentence, "").strip()
    return description
# Read the CSV file and extract the product feature requests
requests = []
with open(csv_file, 'r', encoding='utf-8-sig') as file:
    reader = csv.reader(file)
    for row in reader:
        request_id = row[0]  # Assuming the request id is in the first column
        description = row[16]  # Assuming the description is in the second column
        requests.append((request_id, description))

# Initialize the SentenceTransformer model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

batch_size = 50
records: List[Tuple[str, np.ndarray, Dict]] = []

# Process the dataset in chunks
for chunk_ix, chunk in tqdm(enumerate(flu(requests).chunk(batch_size)), desc="Processing chunks"):
    # Extract just the descriptions from the current chunk
    chunk_descriptions = [preprocess_description(description) for _, description in chunk]
    # Create embeddings for the current chunk
    embedding_chunk = model.encode(chunk_descriptions)

    # Enumerate the embeddings and create a record to insert into the database
    for row_ix, embedding in enumerate(embedding_chunk):
        actual_request_id = chunk[row_ix][0]  # Get the actual request_id from the chunk
        records.append((actual_request_id, embedding, {"text": chunk_descriptions[row_ix]}))

# Insert the records into the collection
for record in tqdm(records, desc="Inserting records"):
    product_requests.upsert([record])

print("All product feature requests have been embedded and inserted into the collection.")

product_requests.create_index()


Compares Vectors by 
connecting to a Supabase database, fetches vector data, calculates the cosine similarity between vectors, filters results based on a similarity threshold, and saves the filtered data to a CSV file.

In [ ]:

# Replace with your Supabase connection details
SUPABASE_URL = os.getenv('SUPABASE_URL')
SUPABASE_KEY = os.getenv('SUPABASE_KEY')

# Define the Supabase client
supabase_client = supabase.create_client(SUPABASE_URL, SUPABASE_KEY)
print("Connected to Supabase")

# Fetch all data from Supabase in one call
try:
    response = supabase_client.table('new').select('id', 'vec').execute()
    data = response.data
    print(f"Fetched {len(data)} rows from Supabase")
except Exception as e:
    print(f"Error fetching data from Supabase: {e}")
    exit()

# Extract vectors and IDs
ids = [item['id'] for item in data]
vectors = [item['vec'] for item in data]

# Convert vectors from strings to lists of floats
try:
    vectors = [ast.literal_eval(vec) for vec in vectors]
    print("Vectors converted from strings to lists")
except Exception as e:
    print(f"Error converting vectors from strings to lists: {e}")

# Convert vectors to numpy array
try:
    vectors = np.array(vectors, dtype=float)
    print("Vectors converted to numpy array")
except Exception as e:
    print(f"Error converting vectors to numpy array: {e}")

# Calculate cosine similarity matrix
try:
    similarity_matrix = cosine_similarity(vectors)
    print("Cosine similarity matrix calculated")
except Exception as e:
    print(f"Error calculating cosine similarity: {e}")

# Minimum similarity threshold (in percentage)
MIN_SIMILARITY_THRESHOLD = 70

# Prepare the data for CSV with similarity percentages and apply threshold
csv_data = []
for idx, feature_id in enumerate(ids):
    similarity_scores = similarity_matrix[idx]
    top_indices = similarity_scores.argsort()[-6:-1][::-1]  # Exclude self and get top 5
    for i in top_indices:
        similarity_percentage = (similarity_scores[i] + 1) / 2 * 100
        if similarity_percentage >= MIN_SIMILARITY_THRESHOLD:
            csv_data.append({
                'Feature ID': feature_id,
                'Similar Feature ID': ids[i],
                'Similarity Percentage': similarity_percentage
            })
    print(f"Top similar features for feature ID {feature_id} found and filtered by similarity threshold")

# Create DataFrame and save to CSV
df = pd.DataFrame(csv_data)
df.to_csv('full_similar_product_features.csv', index=False)
print("CSV file 'full_similar_product_features.csv' created successfully")


Merge similar requests with external Metadata from influence 

In [ ]:

# Paths to the CSV files
full_similar_features_csv = 'similar_product_features.csv'
sap_analytics_csv = 'SAP Analytics Cloud & SAP Digital Boardroom.csv'

try:
    # Load CSV files into Pandas DataFrames
    full_similar_features = pd.read_csv(full_similar_features_csv)
    sap_analytics = pd.read_csv(sap_analytics_csv, usecols=['Request_ID', 'Idea_Title', 'Description','Status'])
except FileNotFoundError:
    print(f"Error: One or more CSV files not found. Please check the file paths.")
    exit(1)
except pd.errors.ParserError:
    print(f"Error: Failed to parse the CSV files. Please check the file format.")
    exit(1)

try:
    # Merge the metadata for both feature_id and similar_feature_id
    merged_data = pd.merge(full_similar_features, sap_analytics, left_on='Feature_ID', right_on='Request_ID')
    merged_data = pd.merge(merged_data, sap_analytics, left_on='Similar_Feature_ID', right_on='Request_ID')
except KeyError:
    print(f"Error: One or more columns not found in the CSV files. Please check the column names.")
    exit(1)

try:
    # Drop the duplicate columns
    merged_data = merged_data.drop_duplicates(subset=['Feature_ID', 'Similar_Feature_ID'], keep='first')
except ValueError:
    print(f"Error: Failed to drop duplicate rows. Please check the data.")
    exit(1)

try:
    # Save the merged data to a new CSV file
    merged_data.to_csv('merged_data.csv', index=False)
except PermissionError:
    print(f"Error: Failed to write to the output CSV file. Please check the file permissions.")
    exit(1)
except IOError:
    print(f"Error: Failed to write to the output CSV file. Please check the file path.")
    exit(1)

print("Data merging completed successfully!")